In [7]:
"""
to open pip "C:\Program Files\Python39\Scripts"
"""
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from random import random
from skimage import exposure


In [8]:

def border_image(original_image, border_size, border_color = (250, 250, 250)):
    # add border
    bordered_image = cv.copyMakeBorder(
                 original_image, 
                 border_size, 
                 border_size, 
                 border_size, 
                 border_size, 
                 cv.BORDER_CONSTANT, 
                 value=border_color
              )
    return bordered_image
    
def make_mask(radius, bordered_image):
    mask = np.zeros(bordered_image.shape, dtype=np.uint8)
    mask_center = tuple(np.array(bordered_image.shape[1::-1]) // 2)
    mask = cv.circle(mask, mask_center, int(radius), (255, 255, 255), -1)
    return mask


def rotate_image(original_image, angle):
    # rotate image
    image_center = tuple(np.array(original_image.shape[1::-1]) / 2)
    rot_mat = cv.getRotationMatrix2D(image_center, angle, 1.0)
    rotated_image = cv.warpAffine(np.copy(original_image), rot_mat, original_image.shape[1::-1], flags=cv.INTER_LINEAR)
    return rotated_image

def apply_background_cover(backgroun_color, background_mask, rotated_image):
    # build background
    background_image = np.full_like(background_mask, backgroun_color)
    background_image = cv.bitwise_or(background_mask, background_image)
    white_bg_rotated_image = cv.bitwise_or(rotated_image, cv.bitwise_not(background_mask))

    return cv.bitwise_and(white_bg_rotated_image, background_image)

def apply_circle_image_border(original_image, original_border, border_color = (250, 250, 250), border_size = 7):
    # add border
    image_center = tuple(np.array(original_image.shape[1::-1]) // 2)
    bordered_image = cv.circle(original_image, image_center, image_center[1]-original_border-1, border_color, border_size)

    return bordered_image

def apply_strange_filter(original_img, color_shift = [15, 0, 13]):
    return cv.add(
        original_img,
        np.full_like(original_img, color_shift)
    )





In [9]:
base_img = cv.imread("./new_gif/stickerino.jpeg")
base_img = cv.cvtColor(base_img, cv.COLOR_BGR2RGB )


In [10]:
def process_image(base_img, angle, radius = None, border_size = 20, backgroun_color = (254, 243, 240), border_color = (255, 1, 1), circle_border_size = 0):
    img = np.copy(base_img)
    img = border_image(img, border_size)
    mask = make_mask(radius if radius else base_img.shape[1]//2, img)
    img = apply_strange_filter(img)
    img = rotate_image(img, angle)
    img = apply_background_cover(backgroun_color, mask, img)
    if circle_border_size > 0:
        img = apply_circle_image_border(img, border_size, border_color, circle_border_size)
    return img

In [29]:
BORDER_SIZE = 50
BACKGROUND_COLOR = (250, 240, 240)

INITIAL_ANGLE = 0
ROTATIONS = 2
DEEGREE_PER_FRAME = 1
FPS = 60


est_duration = (360*ROTATIONS)/(DEEGREE_PER_FRAME*FPS)
print(f"Estimated duration: {est_duration} seconds")

frames = []
for degree in np.arange(INITIAL_ANGLE, INITIAL_ANGLE+int(360*ROTATIONS), DEEGREE_PER_FRAME):
  sim_modifier = (np.sin(degree*0.08) / 55) +0.98
  frames.append(
    process_image(
      base_img, degree, (base_img.shape[1]//2)*sim_modifier ,
      border_size = int(BORDER_SIZE ),
      backgroun_color = np.asarray((BACKGROUND_COLOR[0] * (sim_modifier * 0.53), BACKGROUND_COLOR[1] * (sim_modifier * 0.51), BACKGROUND_COLOR[2] * (sim_modifier * 0.52))),
    )
  )



Estimated duration: 12.0 seconds
Sim modifier: 318.5
Sim modifier: 318.97222319163603
Sim modifier: 319.44142576635693
Sim modifier: 319.90460642888763
Sim modifier: 320.3588024036407
Sim modifier: 320.8011083863693
Sim modifier: 321.22869512819966
Sim modifier: 321.63882753316886
Sim modifier: 322.02888215350504
Sim modifier: 322.3963639707405
Sim modifier: 322.73892235531537
Sim modifier: 323.0543661025848
Sim modifier: 323.3406774490514
Sim modifier: 323.5960249791652
Sim modifier: 323.81877534013387
Sim modifier: 324.0075036898063
Sim modifier: 324.16100281079997
Sim modifier: 324.27829083257234
Sim modifier: 324.3586175120418
Sim modifier: 324.401469032583
Sim modifier: 324.40657129069984
Sim modifier: 324.3738916493523
Sim modifier: 324.30363914672233
Sim modifier: 324.19626315908266
Sim modifier: 324.0524505263224
Sim modifier: 323.8731211585154
Sim modifier: 323.65942215163534
Sim modifier: 323.4127204500558
Sim modifier: 323.1345941027716
Sim modifier: 322.8268221692697
Sim mo

In [30]:
def frames_to_video(frames, fps, file_name = "video.mp4" ):
    out = cv.VideoWriter(file_name, cv.VideoWriter_fourcc(*'mp4v'), int(fps), (frames[0].shape[1], frames[0].shape[0]))
    for frame in frames:
        frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
        out.write(frame)
    out.release()

frames_to_video(frames, FPS)